In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load data from CSV files
train_data = pd.read_csv('mnist_train.csv')
test_data = pd.read_csv('mnist_test.csv')

# Extract features and labels
x_train = train_data.iloc[:, 1:].values / 255.0  # Normalize pixel values
y_train = train_data.iloc[:, 0].values
x_test = test_data.iloc[:, 1:].values / 255.0
y_test = test_data.iloc[:, 0].values



In [ ]:
# Flatten input
x_train, x_test = x_train.reshape(-1, 28*28), x_test.reshape(-1, 28*28)

# Define function to build models
def build_model(regularizer=None):
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizer),
        keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizer),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train baseline model (No Regularization)
baseline_model = build_model()
baseline_history = baseline_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=2)

# Train L1 Regularized Model
l1_model = build_model(keras.regularizers.l1(0.01))
l1_history = l1_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=2)

# Train L2 Regularized Model
l2_model = build_model(keras.regularizers.l2(0.01))
l2_history = l2_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=2)

# Plot Loss and Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(baseline_history.history['loss'], label='Baseline Loss')
plt.plot(l1_history.history['loss'], label='L1 Loss')
plt.plot(l2_history.history['loss'], label='L2 Loss')
plt.title('Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(baseline_history.history['accuracy'], label='Baseline Accuracy')
plt.plot(l1_history.history['accuracy'], label='L1 Accuracy')
plt.plot(l2_history.history['accuracy'], label='L2 Accuracy')
plt.title('Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# Weight distribution visualization
def plot_weights(model, title):
    weights = model.get_weights()[0].flatten()
    plt.hist(weights, bins=50, alpha=0.7)
    plt.title(title)
    plt.xlabel("Weight Values")
    plt.ylabel("Frequency")
    plt.show()

plot_weights(baseline_model, "Baseline Model Weights")
plot_weights(l1_model, "L1 Regularized Weights")
plot_weights(l2_model, "L2 Regularized Weights")
